# Train Baseline UNET model with Single Satellite Image

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
from time import time

In [15]:
from torch.utils.data import DataLoader

In [19]:
from biomasstry.data import Sentinel2

In [3]:
metadata_file = "/notebooks/data/metadata_parquet/features_metadata_slim.parquet"

In [9]:
sen2dataset = Sentinel2(metadata_file)

In [12]:
start = time()
idx_data = sen2dataset[0]
end = time()

In [13]:
print(f"Read time: {end - start}")

Read time: 23.417248487472534


In [16]:
sen2loader = DataLoader(sen2dataset, batch_size=4, num_workers=8)

In [18]:
start = time()
batch_data = next(iter(sen2loader))
end = time()

KeyboardInterrupt: 

In [ ]:
print(f"Read time: {end - start}")